In [1]:
import sys
print(sys.executable)
print(sys.version)

/Library/Developer/CommandLineTools/usr/bin/python3
3.9.6 (default, Oct 17 2025, 17:15:53) 
[Clang 17.0.0 (clang-1700.4.4.1)]


In [2]:
pip install tensorflow-macos==2.15.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow-metal==1.1.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
## 4 classes
import warnings
from tensorflow.keras.preprocessing.image import ImageDataGenerator

warnings.filterwarnings("ignore")

train_dir = "../week3/alzheimer dataset/train"
test_dir  = "../week3/alzheimer dataset/test"


## object for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale= 1/.255, ##normalize pixel values
    shear_range = 0.4 , ## slant range, titling medical images might harm its quality
    zoom_range = 0.2 , ## zoom range for scale 
    width_shift_range = 0.2 , ##moving image left to right
    height_shift_range = 0.2, ##moving image top to bottom
    rotation_range = 90 , ## this might harm medical image classification
    horizontal_flip =True,
    fill_mode = 'nearest'
)

test_datagen = ImageDataGenerator(rescale =1/.255)

##load the images ,labels , ensuring grayscale (1 color channel)
train_generator = train_datagen.flow_from_directory(
    train_dir ,
    target_size =(128, 128), ##resize the images 
    color_mode ='grayscale',
    class_mode='categorical',
    
) # default batch size =32

test_generator =test_datagen.flow_from_directory(
    test_dir,
    target_size=(128,128),
    color_mode ='grayscale',
    class_mode='categorical'
)


/Users/sheenachanda/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Found 4649 images belonging to 4 classes.
Found 1813 images belonging to 4 classes.


In [5]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input , Concatenate ,Flatten ,Dense , Dropout,Conv2D,MaxPooling2D
from tensorflow.keras.models import Model ,Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

Build 3 CNN Models

In [6]:
input_size = (128,128,1)
num_classes=4

In [7]:
##BAse CNN Model
CNN1 = Sequential()
CNN1.add(Conv2D(256,(3,3),padding='same',activation='relu',input_shape=input_size))
CNN1.add(MaxPooling2D(2,2))

CNN1.add(Conv2D(64,(3,3),padding='same',activation='relu'))
CNN1.add(MaxPooling2D(2,2))

CNN1.add(Conv2D(256,(3,3),padding='same',activation='relu'))
CNN1.add(MaxPooling2D(2,2))

CNN1.add(Conv2D(128,(3,3),padding='same',activation='relu'))
CNN1.add(MaxPooling2D(2,2))


CNN1.add(Conv2D(256,(3,3),padding='same',activation='relu'))
CNN1.add(MaxPooling2D(2,2))

CNN1.add(Flatten())
CNN1.add(Dense(128,activation='relu'))
CNN1.add(Dropout(0.2))
CNN1.add(Dense(64,activation='relu'))
CNN1.add(Dense(num_classes,activation='softmax'))


CNN1.compile(
    optimizer= Adam(learning_rate = 0.01),
    loss= 'categorical_crossentropy',
    metrics=['accuracy']
)

CNN1.summary()

CNN1.fit(train_generator,epochs=50,validation_data=test_generator)

2026-02-16 21:24:34.850631: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2026-02-16 21:24:34.850655: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2026-02-16 21:24:34.850663: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.88 GB
2026-02-16 21:24:34.850695: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-02-16 21:24:34.850711: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 256)     2560      
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 256)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 64)        147520    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 32, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 256)       147712    
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 16, 16, 256)       0

2026-02-16 21:24:35.469750: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2026-02-16 21:24:35.500459: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


146/146 [==============================] - 14s 89ms/step - loss: 5106.6221 - accuracy: 0.5489 - val_loss: 1.3192 - val_accuracy: 0.2438
Epoch 2/50
146/146 [==============================] - 13s 87ms/step - loss: 1.2234 - accuracy: 0.5158 - val_loss: 1.5024 - val_accuracy: 0.2438
Epoch 3/50
146/146 [==============================] - 12s 83ms/step - loss: 1.0740 - accuracy: 0.5429 - val_loss: 1.3561 - val_accuracy: 0.2438
Epoch 4/50
146/146 [==============================] - 12s 81ms/step - loss: 1.0319 - accuracy: 0.5704 - val_loss: 1.4739 - val_accuracy: 0.2438
Epoch 5/50
146/146 [==============================] - 12s 79ms/step - loss: 1.0115 - accuracy: 0.5844 - val_loss: 1.2571 - val_accuracy: 0.2438
Epoch 6/50
146/146 [==============================] - 11s 78ms/step - loss: 0.9938 - accuracy: 0.5881 - val_loss: 1.5303 - val_accuracy: 0.2438
Epoch 7/50
146/146 [==============================] - 11s 78ms/step - loss: 0.9959 - accuracy: 0.5909 - val_loss: 1.3300 - val_accuracy: 0.2438


In [8]:
loss,acc= CNN1.evaluate(test_generator)
print(f"Accuracy :{acc : .4f}")

57/57 [==============================] - 6s 109ms/step - loss: 1.4276 - accuracy: 0.2438
Accuracy : 0.2438


In [9]:
##SEcond CNN Model

##BAse CNN Model
CNN2 = Sequential()
CNN2.add(Conv2D(64,(3,3),padding='same',activation='relu',input_shape=input_size))
CNN2.add(MaxPooling2D(2,2))

CNN2.add(Conv2D(64,(3,3),padding='same',activation='relu'))
CNN2.add(MaxPooling2D(2,2))

CNN2.add(Conv2D(128,(3,3),padding='same',activation='relu'))
CNN2.add(MaxPooling2D(2,2))

CNN2.add(Conv2D(128,(3,3),padding='same',activation='relu'))
CNN2.add(MaxPooling2D(2,2))


CNN2.add(Conv2D(256,(3,3),padding='same',activation='relu'))
CNN2.add(MaxPooling2D(2,2))

CNN2.add(Flatten())
CNN2.add(Dense(128,activation='relu'))
CNN2.add(Dropout(0.2))
CNN2.add(Dense(64,activation='relu'))
CNN2.add(Dense(num_classes,activation='softmax'))


CNN2.compile(
    optimizer= Adam(learning_rate = 0.01),
    loss= 'categorical_crossentropy',
    metrics=['accuracy']
)

CNN2.summary()

CNN2.fit(train_generator,epochs=50,validation_data=test_generator)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 128, 128, 64)      640       
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 64, 64, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 64, 64, 64)        36928     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 32, 32, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 32, 32, 128)       73856     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 16, 16, 128)      

2026-02-17 08:59:32.918748: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


146/146 [==============================] - 1671s 5s/step - loss: 2069.2012 - accuracy: 0.4799 - val_loss: 1.3120 - val_accuracy: 0.2438
Epoch 2/50
146/146 [==============================] - 34s 231ms/step - loss: 0.9822 - accuracy: 0.5935 - val_loss: 1.3705 - val_accuracy: 0.2438
Epoch 3/50
146/146 [==============================] - 613s 4s/step - loss: 0.9825 - accuracy: 0.5935 - val_loss: 1.3338 - val_accuracy: 0.2438
Epoch 4/50
146/146 [==============================] - 30s 207ms/step - loss: 0.9802 - accuracy: 0.5935 - val_loss: 1.3108 - val_accuracy: 0.2438
Epoch 5/50
146/146 [==============================] - 1281s 9s/step - loss: 0.9827 - accuracy: 0.5935 - val_loss: 1.3240 - val_accuracy: 0.2438
Epoch 6/50
146/146 [==============================] - 22s 152ms/step - loss: 0.9810 - accuracy: 0.5935 - val_loss: 1.4279 - val_accuracy: 0.2438
Epoch 7/50
146/146 [==============================] - 80s 547ms/step - loss: 0.9835 - accuracy: 0.5935 - val_loss: 1.4193 - val_accuracy: 0.24

In [10]:
loss,acc= CNN2.evaluate(test_generator)
print(f"Accuracy :{acc : .4f}")

57/57 [==============================] - 1s 12ms/step - loss: 1.3641 - accuracy: 0.2438
Accuracy : 0.2438


In [11]:
##SEcond CNN Model

##BAse CNN Model
CNN3 = Sequential()
CNN3.add(Conv2D(64,(3,3),padding='same',activation='relu',input_shape=input_size))
CNN3.add(MaxPooling2D(2,2))

CNN3.add(Conv2D(64,(3,3),padding='same',activation='relu'))
CNN3.add(MaxPooling2D(2,2))

CNN3.add(Conv2D(128,(3,3),padding='same',activation='relu'))
CNN3.add(MaxPooling2D(2,2))

CNN3.add(Conv2D(128,(3,3),padding='same',activation='relu'))
CNN3.add(MaxPooling2D(2,2))


CNN3.add(Conv2D(128,(3,3),padding='same',activation='relu'))
CNN3.add(MaxPooling2D(2,2))

CNN3.add(Flatten())
CNN3.add(Dense(128,activation='relu'))
CNN3.add(Dropout(0.2))
CNN3.add(Dense(64,activation='relu'))
CNN3.add(Dense(num_classes,activation='softmax'))


CNN3.compile(
    optimizer= Adam(learning_rate = 0.02),
    loss= 'categorical_crossentropy',
    metrics=['accuracy']
)

CNN3.summary()

CNN3.fit(train_generator,epochs=50,validation_data=test_generator)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 128, 128, 64)      640       
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 64, 64, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 64, 64, 64)        36928     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 32, 32, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_12 (Conv2D)          (None, 32, 32, 128)       73856     
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 16, 16, 128)      

2026-02-17 10:46:14.421031: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


146/146 [==============================] - 5s 34ms/step - loss: 46632.5000 - accuracy: 0.5399 - val_loss: 1.3976 - val_accuracy: 0.2438
Epoch 2/50
146/146 [==============================] - 5s 35ms/step - loss: 1.0358 - accuracy: 0.5625 - val_loss: 1.3760 - val_accuracy: 0.2438
Epoch 3/50
146/146 [==============================] - 5s 37ms/step - loss: 1.0580 - accuracy: 0.5517 - val_loss: 1.4234 - val_accuracy: 0.2438
Epoch 4/50
146/146 [==============================] - 5s 35ms/step - loss: 1.0564 - accuracy: 0.5670 - val_loss: 1.4184 - val_accuracy: 0.2438
Epoch 5/50
146/146 [==============================] - 5s 36ms/step - loss: 1.0350 - accuracy: 0.5707 - val_loss: 1.5910 - val_accuracy: 0.2438
Epoch 6/50
146/146 [==============================] - 5s 31ms/step - loss: 1.0196 - accuracy: 0.5851 - val_loss: 1.3569 - val_accuracy: 0.2438
Epoch 7/50
146/146 [==============================] - 4s 29ms/step - loss: 1.0140 - accuracy: 0.5797 - val_loss: 1.3723 - val_accuracy: 0.2438
Epoch 

In [12]:
loss,acc= CNN3.evaluate(test_generator)
print(f"Accuracy :{acc : .4f}")

57/57 [==============================] - 1s 11ms/step - loss: 1.3710 - accuracy: 0.2438
Accuracy : 0.2438


### Ensemble Model

In [ ]:
for base_model in [CNN1,CNN2,CNN3]:
    base_model.trainable= False

##shared input

stack_input= Input(shape=input_size)

#get softmax outputs from each base model
pred1 =CNN1(stack_input)
pred2= CNN2(stack_input)
pred3 = CNN3(stack_input)

##concatenate predictions -meta features
merged =Concatenate()([pred1,pred2,pred3]) ##shape (None , 3*n_classes)

meta_input_dim =3*num_classes

MLP_meta = Sequential(name="MLP_meta")
MLP_meta.add(Dense(32,activation ='relu',input_shape=(meta_input_dim,)))                  
MLP_meta.add(Dense(32,activation ='relu'))

stack_output = MLP_meta(merged)

Stacking_Model = Model(inputs=stack_input , outputs=stack_output)

Stacking_Model.compile(
    optimizer= Adam(learning_rate = 0.02),
    loss= 'categorical_crossentropy',
    metrics=['accuracy']
)

Stacking_Model.summary()

history_stack=Stacking_Model.fit(train_generator,epochs=3,validation_data=test_generator)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 1)]        0         []                            
                                                                                                  
 sequential (Sequential)     (None, 4)                    1420932   ['input_1[0][0]']             
                                                                                                  
 sequential_1 (Sequential)   (None, 4)                    1087108   ['input_1[0][0]']             
                                                                                                  
 sequential_2 (Sequential)   (None, 4)                    677380    ['input_1[0][0]']             
                                                                                              

In [ ]:
loss,acc= Stacking_Model.evaluate(test_generator)
print(f"Accuracy :{acc : .4f}")